In [19]:
cd C:/Users/PC/Jeehyun/ECE6780/Data/vgg/

C:\Users\PC\Jeehyun\ECE6780\Data\vgg


In [20]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras import applications
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.models import model_from_json

In [29]:
b_train = 498
m_train = 1190
b_test = 100
m_test = 139
b_val = 137
m_val = 178

def gen_name(mag_fact):
    train_dir = 'breast_' + mag_fact + '_split/train'
    val_dir = 'breast_' + mag_fact + '_split/val'
    model_name = 'bottleneck_fc_model_' + mag_fact + '.h5'
    return train_dir, val_dir, model_name

mag_fact = '200'
nb_train_samples =  b_train + m_train
nb_validation_samples =  b_val + m_val
train_data_dir, validation_data_dir, model_name = gen_name(mag_fact)
sgd = optimizers.SGD(lr=0.001, momentum=0.7)

In [30]:
batch_size = 1
epochs = 50

In [31]:
def features():
    img_width, img_height = 224, 224
    
    datagen = ImageDataGenerator(rescale=1./255)

#    model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
    model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
    print('Model loaded.')

    train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    features_train = model.predict_generator(
        train_generator, nb_train_samples//batch_size)
    np.save(open('base_features_train.npy', 'wb'),
            features_train)

    val_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    features_validation = model.predict_generator(
        val_generator, nb_validation_samples//batch_size)
    np.save(open('base_features_validation.npy', 'wb'),
            features_validation)
    
def train_model():
    train_data = np.load(open('base_features_train.npy', 'rb'))
    train_labels = np.array([0] * b_train + [1] * m_train)

    validation_data = np.load(open('base_features_validation.npy', 'rb'))
    validation_labels = np.array([0] * b_val + [1] * m_val)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(optimizer='sgd',
                  loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(model_name)
    
    model_new_name = 'TL_model_' + mag_fact
    model_json = model.to_json()
    with open(model_new_name + '.json', "w") as json_file:
        json_file.write(model_json)
    model.save_weights(model_new_name + '.h5')

In [32]:
features()
train_model()

Model loaded.
Found 1688 images belonging to 2 classes.
Found 315 images belonging to 2 classes.
Train on 1688 samples, validate on 315 samples
Epoch 1/50
1688/1688 [==============================] - 111s 66ms/step - loss: 0.9659 - accuracy: 0.6973 - val_loss: 0.7329 - val_accuracy: 0.5683
Epoch 2/50
1688/1688 [==============================] - 110s 65ms/step - loss: 0.6283 - accuracy: 0.7026 - val_loss: 0.7374 - val_accuracy: 0.5683
Epoch 3/50
1688/1688 [==============================] - 109s 64ms/step - loss: 0.6111 - accuracy: 0.7073 - val_loss: 0.7060 - val_accuracy: 0.5683
Epoch 4/50
1688/1688 [==============================] - 80s 48ms/step - loss: 0.5893 - accuracy: 0.7150 - val_loss: 0.6365 - val_accuracy: 0.6254
Epoch 5/50
1688/1688 [==============================] - 90s 53ms/step - loss: 0.6056 - accuracy: 0.7085 - val_loss: 0.7062 - val_accuracy: 0.5683
Epoch 6/50
1688/1688 [==============================] - 109s 64ms/step - loss: 0.6015 - accuracy: 0.7103 - val_loss: 0.7130

In [9]:
model_new_name = 'TL_model_' + mag_fact

# load json and create model
json_file = open(model_new_name + '.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(model_new_name + '.h5')
print("Loaded model from disk")

Loaded model from disk


In [10]:
batch_size = 20
epochs = 50
loaded_model.compile(optimizer='sgd',loss='binary_crossentropy', metrics=['accuracy'])
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               13107456  
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 13,107,713
Trainable params: 13,107,713
Non-trainable params: 0
__________________________________________

In [15]:
train_data = np.load(open('base_features_train.npy', 'rb'))
train_labels = np.array([0] * b_train + [1] * m_train)

validation_data = np.load(open('base_features_validation.npy', 'rb'))
validation_labels = np.array([0] * b_val + [1] * m_val)

loaded_model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))

Train on 1688 samples, validate on 315 samples
Epoch 1/50
1688/1688 [==============================] - 4s 3ms/step - loss: 0.6038 - accuracy: 0.7073 - val_loss: 0.7293 - val_accuracy: 0.5651
Epoch 2/50
1688/1688 [==============================] - 4s 2ms/step - loss: 0.6036 - accuracy: 0.7073 - val_loss: 0.7270 - val_accuracy: 0.5651
Epoch 3/50
1688/1688 [==============================] - 4s 3ms/step - loss: 0.5837 - accuracy: 0.7251 - val_loss: 0.6324 - val_accuracy: 0.6508
Epoch 4/50
1688/1688 [==============================] - 4s 2ms/step - loss: 0.5342 - accuracy: 0.7648 - val_loss: 0.6099 - val_accuracy: 0.6762
Epoch 5/50
1688/1688 [==============================] - 4s 2ms/step - loss: 0.5033 - accuracy: 0.7927 - val_loss: 0.4955 - val_accuracy: 0.7810
Epoch 6/50
1688/1688 [==============================] - 4s 2ms/step - loss: 0.4855 - accuracy: 0.8110 - val_loss: 0.4862 - val_accuracy: 0.7841
Epoch 7/50
1688/1688 [==============================] - 4s 2ms/step - loss: 0.4616 - accu

In [18]:
model_new_name = 'TL_model' + mag_fact
model_json = loaded_model.to_json()

with open(model_new_name + '.json', "w") as json_file:
    json_file.write(model_json)
    loaded_model.save_weights(model_new_name + '.h5')